# DGXB Data Exploration

This notebook explores the traffic and weather data from the bronze and silver layers.

In [36]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import sys

# Add dgxb to path
sys.path.insert(0, str(Path.cwd() / 'dgxb'))

# Set up plotting
%matplotlib inline
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


In [38]:
# Load silver layer traffic data
from dgxb.etl.silver_processor import load_bronze_parquets, clean_traffic_data

# Try loading from silver first, then bronze if needed
silver_traffic_path = Path("silver-cpu-traffic/data_silver.parquet")
if silver_traffic_path.exists():
    print("Loading from silver layer...")
    traffic_df = pd.read_parquet(silver_traffic_path)
else:
    print("Silver layer not found, processing from bronze...")
    bronze_df = load_bronze_parquets("bronze-traffic", file_pattern="traffic_*.parquet")
    traffic_df = clean_traffic_data(bronze_df)

print(f"Traffic data loaded: {len(traffic_df)} records")
print(f"Columns: {list(traffic_df.columns)}")
traffic_df

Loading from silver layer...
Traffic data loaded: 3956 records
Columns: ['incident_id', 'timestamp', 'description', 'location', 'lat', 'lon', 'address', 'status', 'traffic_report_status_date_time', 'agency', 'location_type', 'location_lon', 'location_lat']


,incident_id,timestamp,description,location,lat,lon,address,status,traffic_report_status_date_time,agency,location_type,location_lon,location_lat
0,BD8C106CC7D7649BEF42E9662D7E821A6A2961E8_17630...,2025-11-13 03:50:29+00:00,COLLISION WITH INJURY,"{'coordinates': [-97.630978, 30.423636], 'type...",30.423636,-97.630978,1415 ST CROIX LN,ARCHIVED,2025-11-13T05:30:18.000Z,TRAVIS COUNTY SHERIF,Point,-97.630978,30.423636
1,38165AF29AB1E2D307F60404EC6C2DB11E51C84E_17630...,2025-11-13 04:46:03+00:00,Stalled Vehicle,"{'coordinates': [-97.787383, 30.229102], 'type...",30.229102,-97.787383,4400 Merle Dr,ARCHIVED,2025-11-13T05:15:16.000Z,AUSTIN PD,Point,-97.787383,30.229102
2,C718AA30A6FF882B7087CE4F1567E335EA77186E_17630...,2025-11-13 05:30:57+00:00,Crash Service,"{'coordinates': [-97.789515, 30.165489], 'type...",30.165489,-97.789515,9300 S Ih 35 Svrd Sb,ARCHIVED,2025-11-13T05:45:19.000Z,AUSTIN PD,Point,-97.789515,30.165489
3,BFAC6A4B8FAC8CC5337C3E58459324C560CFEA67_17630...,2025-11-13 06:11:25+00:00,Crash Urgent,"{'coordinates': [-97.659533, 30.212871], 'type...",30.212871,-97.659533,0 E 71 Svrd Eb To President Ramp,ARCHIVED,2025-11-13T07:15:15.000Z,AUSTIN PD,Point,-97.659533,30.212871
4,E1E2113D3105A4AB5E0052F7803AFC0D1079B55C_17630...,2025-11-13 07:29:23+00:00,Crash Urgent,"{'coordinates': [-97.719775, 30.29498], 'type'...",30.294980,-97.719775,N Ih 35 Svrd Sb / E 38th Half St,ARCHIVED,2025-11-13T09:45:22.000Z,AUSTIN PD,Point,-97.719775,30.294980
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3951,1D23C8EDF135106A57D7F1694D9C2C7EB67CC326_17655...,2025-12-13 03:08:35+00:00,Crash Urgent,"{'coordinates': [-97.854643, 30.186248], 'type...",30.186248,-97.854643,3815 W Slaughter Ln,ACTIVE,2025-12-13T03:10:14.000Z,AUSTIN PD,Point,-97.854643,30.186248
3952,5991C2EFCFB90EC6BC16FE7A1100F5F49A2EE4A1_17655...,2025-12-13 03:12:42+00:00,COLLISION,"{'coordinates': [-97.666111, 30.453229], 'type...",30.453229,-97.666111,15700 N Ih 35 Nb,ACTIVE,2025-12-13T03:15:16.000Z,TRAVIS COUNTY SHERIF,Point,-97.666111,30.453229
3953,51C86E5A56854DBD8F91BCB9B15CF18A2F766032_17655...,2025-12-13 03:18:17+00:00,TRFC HAZD/ DEBRIS,"{'coordinates': [-97.580822, 30.500364], 'type...",30.500364,-97.580822,20219-20625 N Sh 130 Nb,ACTIVE,2025-12-13T03:25:16.000Z,TRAVIS COUNTY SHERIF,Point,-97.580822,30.500364
3954,CA9E263D07340B8B04EEA95E4172CCB3800E773A_17655...,2025-12-13 03:24:09+00:00,Traffic Hazard,"{'coordinates': [-97.730962, 30.274694], 'type...",30.274694,-97.730962,1413-1500 N Ih 35 Svrd Nb,ARCHIVED,2025-12-13T03:40:14.000Z,AUSTIN PD,Point,-97.730962,30.274694


In [40]:
# Load silver layer weather data
silver_weather_path = Path("silver-cpu-weather/data_silver.parquet")
if silver_weather_path.exists():
    print("Loading from silver layer...")
    weather_df = pd.read_parquet(silver_weather_path)
else:
    print("Silver layer not found, processing from bronze...")
    from dgxb.etl.silver_processor import clean_weather_data
    bronze_df = load_bronze_parquets("bronze-weather", file_pattern="weather_*.parquet")
    weather_df = clean_weather_data(bronze_df)

print(f"Weather data loaded: {len(weather_df)} records")
print(f"Columns: {list(weather_df.columns)}")
weather_df

Loading from silver layer...
Weather data loaded: 3742 records
Columns: ['timestamp', 'lat', 'lon', 'temperature', 'dewpoint', 'humidity', 'wind_speed', 'wind_direction', 'precipitation_amount', 'precipitation_probability', 'weather_code', 'data_source', 'short_forecast', 'detailed_forecast', 'is_daytime']


,timestamp,lat,lon,temperature,dewpoint,humidity,wind_speed,wind_direction,precipitation_amount,precipitation_probability,weather_code,data_source,short_forecast,detailed_forecast,is_daytime
0,2025-11-13 00:00:00+00:00,30.2672,-97.7431,18.0,15.200000,84,10.8,182.0,0.0,NaN,0.0,historical,None,None,None
1,2025-11-13 00:00:00+00:00,30.2244,-97.7694,17.5,15.100000,86,12.7,186.0,0.0,NaN,0.0,historical,None,None,None
2,2025-11-13 00:00:00+00:00,30.2746,-97.7406,17.9,15.200000,84,10.8,182.0,0.0,NaN,0.0,historical,None,None,None
3,2025-11-13 00:00:00+00:00,30.3072,-97.7559,17.2,15.200000,88,10.5,174.0,0.0,NaN,0.0,historical,None,None,None
4,2025-11-13 00:00:00+00:00,30.2849,-97.7341,17.9,15.200000,84,10.8,182.0,0.0,NaN,0.0,historical,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3737,2025-12-14 23:00:00+00:00,30.2672,-97.7431,53.0,2.222222,52,10.0,NaN,NaN,0.0,NaN,None,Mostly Sunny,,True
3738,2025-12-15 00:00:00+00:00,30.2672,-97.7431,51.0,1.666667,54,10.0,NaN,NaN,0.0,NaN,None,Partly Cloudy,,False
3739,2025-12-15 01:00:00+00:00,30.2672,-97.7431,50.0,1.111111,54,10.0,NaN,NaN,0.0,NaN,None,Partly Cloudy,,False
3740,2025-12-15 02:00:00+00:00,30.2672,-97.7431,48.0,1.111111,58,5.0,NaN,NaN,0.0,NaN,None,Partly Cloudy,,False


In [42]:
# Load gold layer data (X features and y target)
X_path = Path("gold-cpu-traffic/X_features.parquet")
y_path = Path("gold-cpu-traffic/y_target.parquet")

if X_path.exists() and y_path.exists():
    X_features = pd.read_parquet(X_path)
    y_target = pd.read_parquet(y_path)
    print(f"Loaded X: {len(X_features):,} records, {len(X_features.columns)} features")
    print(f"Loaded y: {len(y_target):,} records")
    print(f"\nX features (first 5 rows):")
    X_features.head()
    print(f"\ny target (first 5 rows):")
    y_target.head()
else:
    print("Gold data not found. Run: python dgxb/run_feature_engineering.py")


,incident_id,timestamp,description,location,lat,lon,address,status,traffic_report_status_date_time,agency,...,weather_precipitation_probability,weather_weather_code,weather_data_source,weather_short_forecast,weather_detailed_forecast,weather_is_daytime,weather_distance_km,weather_time_diff_hours,weather_combined_score,weather_h3_cell
0,BD8C106CC7D7649BEF42E9662D7E821A6A2961E8_17630...,2025-11-13 03:50:29+00:00,COLLISION WITH INJURY,"{'coordinates': [-97.630978, 30.423636], 'type...",30.423636,-97.630978,1415 ST CROIX LN,ARCHIVED,2025-11-13T05:30:18.000Z,TRAVIS COUNTY SHERIF,...,NaN,NaN,None,None,None,None,NaN,NaN,NaN,None
1,38165AF29AB1E2D307F60404EC6C2DB11E51C84E_17630...,2025-11-13 04:46:03+00:00,Stalled Vehicle,"{'coordinates': [-97.787383, 30.229102], 'type...",30.229102,-97.787383,4400 Merle Dr,ARCHIVED,2025-11-13T05:15:16.000Z,AUSTIN PD,...,NaN,NaN,None,None,None,None,NaN,NaN,NaN,None
2,C718AA30A6FF882B7087CE4F1567E335EA77186E_17630...,2025-11-13 05:30:57+00:00,Crash Service,"{'coordinates': [-97.789515, 30.165489], 'type...",30.165489,-97.789515,9300 S Ih 35 Svrd Sb,ARCHIVED,2025-11-13T05:45:19.000Z,AUSTIN PD,...,NaN,NaN,None,None,None,None,NaN,NaN,NaN,None
3,BFAC6A4B8FAC8CC5337C3E58459324C560CFEA67_17630...,2025-11-13 06:11:25+00:00,Crash Urgent,"{'coordinates': [-97.659533, 30.212871], 'type...",30.212871,-97.659533,0 E 71 Svrd Eb To President Ramp,ARCHIVED,2025-11-13T07:15:15.000Z,AUSTIN PD,...,NaN,NaN,None,None,None,None,NaN,NaN,NaN,None
4,E1E2113D3105A4AB5E0052F7803AFC0D1079B55C_17630...,2025-11-13 07:29:23+00:00,Crash Urgent,"{'coordinates': [-97.719775, 30.29498], 'type'...",30.294980,-97.719775,N Ih 35 Svrd Sb / E 38th Half St,ARCHIVED,2025-11-13T09:45:22.000Z,AUSTIN PD,...,NaN,NaN,None,None,None,None,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3951,1D23C8EDF135106A57D7F1694D9C2C7EB67CC326_17655...,2025-12-13 03:08:35+00:00,Crash Urgent,"{'coordinates': [-97.854643, 30.186248], 'type...",30.186248,-97.854643,3815 W Slaughter Ln,ACTIVE,2025-12-13T03:10:14.000Z,AUSTIN PD,...,NaN,NaN,None,None,None,None,NaN,NaN,NaN,None
3952,5991C2EFCFB90EC6BC16FE7A1100F5F49A2EE4A1_17655...,2025-12-13 03:12:42+00:00,COLLISION,"{'coordinates': [-97.666111, 30.453229], 'type...",30.453229,-97.666111,15700 N Ih 35 Nb,ACTIVE,2025-12-13T03:15:16.000Z,TRAVIS COUNTY SHERIF,...,NaN,NaN,None,None,None,None,NaN,NaN,NaN,None
3953,51C86E5A56854DBD8F91BCB9B15CF18A2F766032_17655...,2025-12-13 03:18:17+00:00,TRFC HAZD/ DEBRIS,"{'coordinates': [-97.580822, 30.500364], 'type...",30.500364,-97.580822,20219-20625 N Sh 130 Nb,ACTIVE,2025-12-13T03:25:16.000Z,TRAVIS COUNTY SHERIF,...,NaN,NaN,None,None,None,None,NaN,NaN,NaN,None
3954,CA9E263D07340B8B04EEA95E4172CCB3800E773A_17655...,2025-12-13 03:24:09+00:00,Traffic Hazard,"{'coordinates': [-97.730962, 30.274694], 'type...",30.274694,-97.730962,1413-1500 N Ih 35 Svrd Nb,ARCHIVED,2025-12-13T03:40:14.000Z,AUSTIN PD,...,NaN,NaN,None,None,None,None,NaN,NaN,NaN,None


In [30]:
# Feature columns (X)
print(f"X has {len(X_features.columns)} feature columns:")
X_features.columns.tolist()


Index(['incident_id', 'timestamp', 'description', 'location', 'lat', 'lon',
       'address', 'status', 'traffic_report_status_date_time', 'agency',
       'location_type', 'location_lon', 'location_lat', 'h3_cell',
       'weather_temperature', 'weather_dewpoint', 'weather_humidity',
       'weather_wind_speed', 'weather_wind_direction',
       'weather_precipitation_amount', 'weather_precipitation_probability',
       'weather_weather_code', 'weather_data_source', 'weather_short_forecast',
       'weather_detailed_forecast', 'weather_is_daytime',
       'weather_distance_km', 'weather_time_diff_hours',
       'weather_combined_score', 'weather_h3_cell'],
      dtype='object')

In [32]:
# Target variable (y) - incident categories
print(f"Target categories:")
print(y_target['incident_category'].value_counts())
print(f"\nOriginal descriptions (sample):")
y_target[['description', 'incident_category', 'incident_category_confidence']].head(10)


array(['COLLISION WITH INJURY', 'Stalled Vehicle', 'Crash Service',
       'Crash Urgent', 'TRFC HAZD/ DEBRIS', 'Traffic Hazard', 'COLLISION',
       'LOOSE LIVESTOCK', 'COLLISN/ LVNG SCN',
       'COLLISION/PRIVATE PROPERTY', 'VEHICLE FIRE', 'ICY ROADWAY',
       'OBSTRUCT HWY', 'BLOCKED DRIV/ HWY', 'AUTO/ PED'], dtype=object)